In [291]:
%load_ext autotime

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 0 ns


In [292]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
# pd.options.display.max_columns
pd.set_option("display.max_colwidth",200)
pd.set_option("display.max_columns",20)
pd.set_option('float_format', '{:.3f}'.format)

time: 0 ns


In [293]:
import pandas as pd
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
# pd.options.display.max_columns
pd.set_option("display.max_colwidth",200)
pd.set_option("display.max_columns",20)
pd.set_option('float_format', '{:.3f}'.format)

# Load Usage data
with open('App_usage_trace.txt') as f:
    lines = f.readlines()#[:100000]
    df = []
    for line in lines:
        data = line.split()
        df.append(data)
usage = pd.DataFrame(df, columns=['uid','timestamp', 'loc', 'app_id','traffic'])
usage['traffic'] = usage['traffic'].astype('float64') / 1e6 # Convert traffic to MB
usage['timestamp'] = usage['timestamp'].apply(lambda x: datetime.strptime(x, "%Y%m%d%H%M%S")) # Convert to datetime object
usage['loc'] = usage['loc'].astype('int64')
usage["date"] = usage["timestamp"].dt.strftime("%m-%d")
usage.head()

,uid,timestamp,loc,app_id,traffic,date
0,0,2016-04-20 08:13:19,8194,361,0.002,04-20
1,0,2016-04-20 08:13:20,8194,361,0.002,04-20
2,0,2016-04-20 08:13:22,8194,361,0.004,04-20
3,0,2016-04-20 08:13:30,8194,361,0.001,04-20
4,0,2016-04-20 08:13:31,8194,361,0.002,04-20


time: 1min 5s


# Identify users that have less than X records per day, see what is the impact of the removal

In [294]:
# group uid and date, purpose is to see the number of request that a user made for each day
user_usage_perday = usage.groupby(["uid", "date"]).size().reset_index()
# rename columns for easy slicing
user_usage_perday.columns = ["uid", "date", "count"]
# number of users that does not have X counts per day
# change this
required_count = 10

# slice dataset to only remain those rows that has less than 20 count
df_less10day = user_usage_perday[user_usage_perday["count"] < required_count]
# group dataset with uid to get the count of the occurance of the uid, so we can determine which users have just 1 day or more
uid_less10day = df_less10day.groupby(["uid"]).size().reset_index()
# rename columns for easy slicing
uid_less10day.columns = ["uid", "count"]
print("="*100)
print("Current condition: remove user that has less than {} records per day".format(required_count))
print("="*100)
print("There are {} users that does not meet the requirement". format(uid_less10day.shape[0]))
print("-"*100)
print("There are {} users that have less than {} records for exact 1 day".format((uid_less10day.shape[0]- uid_less10day[uid_less10day["count"] != 1].shape[0]), required_count))
print("-"*100)
print("There are {} users that have less than {} records for more than 1 day".format(uid_less10day[uid_less10day["count"] != 1].shape[0], required_count))
print("-"*100)
print("Under current condition, {} rows will be removed, {:.5%} of the entire dataset ".format(df_less10day["count"].sum(), df_less10day["count"].sum()/414171950))
print("="*100)

Current condition: remove user that has less than 10 records per day
There are 200 users that does not meet the requirement
----------------------------------------------------------------------------------------------------
There are 140 users that have less than 10 records for exact 1 day
----------------------------------------------------------------------------------------------------
There are 60 users that have less than 10 records for more than 1 day
----------------------------------------------------------------------------------------------------
Under current condition, 1263 rows will be removed, 0.00030% of the entire dataset 
time: 1.08 s


# Implementation of the cleaning

In [5]:
required_count = 10
# slice dataset to only remain those rows that has less than 20 count
user_usage_perday = usage.groupby(["uid", "date"]).size().reset_index()
user_usage_perday.columns = ["uid", "date", "count"]

df_less10day = user_usage_perday[user_usage_perday["count"] < required_count]
records_remove = np.array(df_less10day)
usage_copy = usage.copy()
print("Before removal the dataset has {} rows".format(usage.shape[0]))
for i in range(len(records_remove)):
    uid = records_remove[i][0]
    date = records_remove[i][1]
    usage_copy = usage_copy.drop(usage_copy[(usage_copy["uid"] == uid) & (usage_copy["date"] == date)].index)
print("After removal the dataset has {} rows, {} rows has been removed".format(usage_copy.shape[0], (usage.shape[0] - usage_copy.shape[0])))

Before removal the dataset has 4171950 rows
After removal the dataset has 4170687 rows, 1263 rows has been removed
time: 5min 3s


In [295]:
# rename the cleaned dataset 
usage_cleaned = usage.copy()

time: 219 ms


# Adpot Oscillation

In [233]:
# Function to align the Origin and Destination (O/D)
def sort_movement_tuples(jump):
    return sorted(list(jump))

# Define a function that can output the frequency pair of a user's mobility pattern
def show_frequent_pair(uid=888, threshold=5, jump_occurence=10):
    # Nathan: replaced usage with usage_cleaned
    user_uid = usage_cleaned[usage_cleaned['uid'] == str(uid)]

    # Set the thresold of time gap allowed between movement
    delta_threshold = timedelta(seconds=threshold)

    # Create the columns of the next time and lcoation
    user_uid['next_loc'] = user_uid['loc'].shift(-1).fillna(0).astype('int')
    user_uid['next_timestamp'] = user_uid['timestamp'].shift(-1)

    # Keep only movements
    user_uid = user_uid[user_uid['loc'] != user_uid['next_loc']]

    # Get the time gaps between movements
    user_uid['time_delta'] = user_uid['next_timestamp'] - user_uid['timestamp']

    # Get teleports and count the teleports
    # Should be able to identify the return trips
    teleports = user_uid[user_uid['time_delta'] < delta_threshold]
    teleports['tele'] = list(zip(teleports['loc'], teleports['next_loc']))
    # print(teleports['tele'].value_counts())

    # # Group and count the same jump
    tele_counts = teleports['tele'].value_counts().reset_index()
    tele_counts.columns = ['jump', 'count']

    # Align the O/D of the jump to identify the pairs
    tele_counts['jump'] = tele_counts['jump'].apply(lambda i: sort_movement_tuples(i))

    # Get the jumps that has a reasonable number of occurence
    tele_counts = tele_counts[tele_counts['count'] > jump_occurence]
    tele_counts['jump'] = tuple(tele_counts['jump'])

    # print the record for reference
    # Nathan: added reset_index()
    frequent_pairs = tele_counts.groupby(['jump']).sum().reset_index()

    return frequent_pairs

time: 0 ns


## First way to solve oscillation:
## 1) get frequent pairs with show_frequent_pair()
## 2) call the oscillation period using present_oscillation()
## 3) from the frequent pairs compare their overall count for that particular user 
## (This can be modify to any condition)
## 4) get the index of roles that requires changing loc
## 5) change loc

In [296]:
# Function to align the Origin and Destination (O/D)
def sort_movement_tuples(jump):
    return sorted(list(jump))

# show the period where the oscillation happens
def present_oscillation(uid=888, threshold=5, jump_occurence=10):
    # Nathan: replaced usage with usage_cleaned
    user_uid = usage_cleaned[usage_cleaned['uid'] == str(uid)]

    # Set the thresold of time gap allowed between movement
    delta_threshold = timedelta(seconds=threshold)

    # Create the columns of the next time and lcoation
    user_uid['next_loc'] = user_uid['loc'].shift(-1).fillna(0).astype('int')
    user_uid['next_timestamp'] = user_uid['timestamp'].shift(-1)

    # Keep only movements
    user_uid = user_uid[user_uid['loc'] != user_uid['next_loc']]

    # Get the time gaps between movements
    user_uid['time_delta'] = user_uid['next_timestamp'] - user_uid['timestamp']

    # Get teleports and count the teleports
    # Should be able to identify the return trips
    teleports = user_uid[user_uid['time_delta'] < delta_threshold]
    teleports['tele'] = list(zip(teleports['loc'], teleports['next_loc']))
    # print(teleports['tele'].value_counts())

    # # Group and count the same jump
    tele_counts = teleports['tele'].value_counts().reset_index()
    tele_counts.columns = ['jump', 'count']

    # Align the O/D of the jump to identify the pairs
    # we want to remove this as we want to return the rows that travel from a>b and b>a
    tele_counts['jump'] = tele_counts['jump'].apply(lambda i: sort_movement_tuples(i))

    # Get the jumps that has a reasonable number of occurence
    tele_counts = tele_counts[tele_counts['count'] > jump_occurence]
    tele_counts['jump'] = tuple(tele_counts['jump'])

    teleports['tele'] = teleports['tele'].apply(lambda j: sort_movement_tuples(j))
    teleports['tele'] = tuple(teleports['tele'])

    #return the rows that has tele_count in the "tele" column, so it returns the rows where the oscillation happens
    return teleports[teleports['tele'].isin(tele_counts["jump"])]
    # return teleports

time: 0 ns


In [249]:
# show the frequent pair
user0_freq_pair = np.array(show_frequent_pair(772,10,5))
user0_freq_pair

<ipython-input-233-20c6b7ee343d>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  user_uid['next_loc'] = user_uid['loc'].shift(-1).fillna(0).astype('int')
<ipython-input-233-20c6b7ee343d>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  user_uid['next_timestamp'] = user_uid['timestamp'].shift(-1)
<ipython-input-233-20c6b7ee343d>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.o

array([[(131, 4595), 7],
       [(131, 5029), 29],
       [(131, 5087), 17],
       ...,
       [(9630, 9800), 6],
       [(9631, 9723), 21],
       [(9631, 9800), 61]], dtype=object)

time: 6.38 s


In [253]:
user0_freq_pair.shape[0]

2453

time: 0 ns


In [250]:
# show the oscillation period
user0_oscillation_period = present_oscillation(772,10,5)
user0_oscillation_period

<ipython-input-245-f1d55fd8c473>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  user_uid['next_loc'] = user_uid['loc'].shift(-1).fillna(0).astype('int')
<ipython-input-245-f1d55fd8c473>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  user_uid['next_timestamp'] = user_uid['timestamp'].shift(-1)
<ipython-input-245-f1d55fd8c473>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.o

,uid,timestamp,loc,app_id,traffic,date,next_loc,next_timestamp,time_delta,tele
1771575,772,2016-04-20 00:05:11,6392,68,0.006,04-20,8803,2016-04-20 00:05:17,00:00:06,"(6392, 8803)"
1771670,772,2016-04-20 07:12:00,9631,2,0.002,04-20,8951,2016-04-20 07:12:01,00:00:01,"(8951, 9631)"
1771671,772,2016-04-20 07:12:01,8951,1,0.015,04-20,9631,2016-04-20 07:12:01,00:00:00,"(8951, 9631)"
1771673,772,2016-04-20 07:12:23,8951,1,0.002,04-20,9631,2016-04-20 07:12:26,00:00:03,"(8951, 9631)"
1771676,772,2016-04-20 07:12:28,9631,2,0.003,04-20,8951,2016-04-20 07:12:34,00:00:06,"(8951, 9631)"
1771677,772,2016-04-20 07:12:34,8951,1,0.003,04-20,9631,2016-04-20 07:12:41,00:00:07,"(8951, 9631)"
1771682,772,2016-04-20 07:14:01,9631,2,0.000,04-20,8951,2016-04-20 07:14:03,00:00:02,"(8951, 9631)"
1771683,772,2016-04-20 07:14:03,8951,286,0.025,04-20,9631,2016-04-20 07:14:04,00:00:01,"(8951, 9631)"
1771684,772,2016-04-20 07:14:04,9631,2,0.000,04-20,8951,2016-04-20 07:14:08,00:00:04,"(8951, 9631)"
1771685,772,2016-04-20 07:14:08,8951,286,0.021,04-20,9631,2016-04-20 07:14:14,00:00:06,"(8951, 9631)"


time: 8.94 s


In [251]:
for i in range(len(user0_freq_pair)):
    loc1 = user0_freq_pair[i][0][0]
    loc2 = user0_freq_pair[i][0][1]

        # any condition
    if usage_cleaned[(usage_cleaned["uid"] == str(0)) & (usage_cleaned["loc"] == loc1)].shape[0] > usage_cleaned[(usage_cleaned["uid"] == str(0)) & (usage_cleaned["loc"] == loc2)].shape[0]:
        # current use print instead of actually replace the loc 
        
        print(user0_oscillation_period[(user0_oscillation_period["tele"] == user0_freq_pair[i][0]) &(user0_oscillation_period["loc"] == loc2)].index)
        print("these index in the original data should be changed to {}".format(loc1))
    else:
        print(user0_oscillation_period[(user0_oscillation_period["tele"] == user0_freq_pair[i][0]) &(user0_oscillation_period["loc"] == loc1)].index)
        print("these index in the original data should be changed to {}".format(loc2))

3,
            1930477, 1930481, 1930499, 1930515, 1930797, 1948449, 1995028],
           dtype='int64')
these index in the original data should be change to 8951
Int64Index([1930600, 1930621, 1930628, 1930634, 1930637, 1930643, 1930646,
            1930650, 1930654, 1930659, 1930663, 1930668, 1930675, 1930680,
            1930683, 1930686, 1930730, 1930734, 1930736, 1930748, 1930768],
           dtype='int64')
these index in the original data should be change to 8960
Int64Index([1791893, 1957469, 1957472, 1957483, 1957490, 1957493, 1957499,
            1957549, 1957559, 1957562, 1957586, 1957731, 1957904, 1957906,
            1957976, 1957994, 1957996, 1958000, 1958263, 1958266, 1958268,
            1958270, 1958273, 1958369, 1958809, 1958812, 1958814, 1958894,
            1958957, 1958989, 1959000],
           dtype='int64')
these index in the original data should be change to 9631
Int64Index([1918701, 1918910, 1918913, 1918916, 1919207, 1919300], dtype='int64')
these index in the or

## Index method 
## 1) get the start index and end index of the oscillation period
## 2) get the count of the stations
## 3) if different, replace a>b or b>a
## 3) if same, increase the counting period by decreasing the start index and increasing the end index until one loc outnumber the other
## 4) ?

In [297]:
user0_freq_pair = np.array(show_frequent_pair(0,10,4))
user0_freq_pair

<ipython-input-233-20c6b7ee343d>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  user_uid['next_loc'] = user_uid['loc'].shift(-1).fillna(0).astype('int')
<ipython-input-233-20c6b7ee343d>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  user_uid['next_timestamp'] = user_uid['timestamp'].shift(-1)
<ipython-input-233-20c6b7ee343d>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.o

array([[(1361, 2304), 14],
       [(2318, 8120), 10],
       [(7031, 7692), 5]], dtype=object)

time: 8.66 s


In [298]:
user0_oscillation_period = present_oscillation(0,10,4)
user0_oscillation_period

<ipython-input-296-f1d55fd8c473>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  user_uid['next_loc'] = user_uid['loc'].shift(-1).fillna(0).astype('int')
<ipython-input-296-f1d55fd8c473>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  user_uid['next_timestamp'] = user_uid['timestamp'].shift(-1)
<ipython-input-296-f1d55fd8c473>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.o

,uid,timestamp,loc,app_id,traffic,date,next_loc,next_timestamp,time_delta,tele
1006,0,2016-04-22 08:45:22,2318,34,0.009,04-22,8120,2016-04-22 08:45:22,00:00:00,"(2318, 8120)"
1008,0,2016-04-22 08:45:22,8120,34,0.003,04-22,2318,2016-04-22 08:45:28,00:00:06,"(2318, 8120)"
1009,0,2016-04-22 08:45:28,2318,29,0.010,04-22,8120,2016-04-22 08:45:28,00:00:00,"(2318, 8120)"
1010,0,2016-04-22 08:45:28,8120,29,0.005,04-22,2318,2016-04-22 08:45:31,00:00:03,"(2318, 8120)"
1016,0,2016-04-22 08:45:45,2318,34,0.001,04-22,8120,2016-04-22 08:45:45,00:00:00,"(2318, 8120)"
1017,0,2016-04-22 08:45:45,8120,34,0.020,04-22,2318,2016-04-22 08:45:46,00:00:01,"(2318, 8120)"
1020,0,2016-04-22 08:45:47,2318,34,0.002,04-22,8120,2016-04-22 08:45:47,00:00:00,"(2318, 8120)"
1021,0,2016-04-22 08:45:47,8120,29,0.026,04-22,2318,2016-04-22 08:45:48,00:00:01,"(2318, 8120)"
1022,0,2016-04-22 08:45:48,2318,29,0.001,04-22,8120,2016-04-22 08:45:48,00:00:00,"(2318, 8120)"
1024,0,2016-04-22 08:45:49,8120,34,0.002,04-22,2318,2016-04-22 08:45:50,00:00:01,"(2318, 8120)"


time: 8.64 s


In [323]:
start_index = 1787
end_index = 1804
oscillation_period = usage_cleaned.loc[start_index:end_index, "loc"].copy()
count_loc1 = oscillation_period[oscillation_period == 1361].shape[0]
count_loc2 = oscillation_period[oscillation_period == 2304].shape[0]

while abs(count_loc1-count_loc2) < 5:

    start_index -=1
    end_index +=1

    oscillation_period = usage_cleaned.loc[start_index:end_index, "loc"].copy()
    count_loc1 = oscillation_period[oscillation_period == 1361].shape[0]
    count_loc2 = oscillation_period[oscillation_period == 2304].shape[0]

    print("start index: {}, end_index = {}, count_loc1:{}, count_loc2:{}".format(start_index, end_index, count_loc1, count_loc2))
    
print("finished")
# if count_loc1 > count_loc2:
#     oscillation_period[oscillation_period == 2304] = 1361
# else:
#     oscillation_period[oscillation_period == 1361] = 2304

# oscillation_period


start index: 1786, end_index = 1805, count_loc1:10, count_loc2:9
start index: 1785, end_index = 1806, count_loc1:11, count_loc2:9
start index: 1784, end_index = 1807, count_loc1:12, count_loc2:9
start index: 1783, end_index = 1808, count_loc1:13, count_loc2:9
start index: 1782, end_index = 1809, count_loc1:14, count_loc2:9
finished
time: 15 ms


In [321]:
for i in range(len(user0_freq_pair)):
    freq_pair = user0_freq_pair[i][0]
    loc1 = user0_freq_pair[i][0][0]
    loc2 = user0_freq_pair[i][0][1]
    

    start_index = user0_oscillation_period[user0_oscillation_period["tele"] == freq_pair].index[0]
    end_index = user0_oscillation_period[user0_oscillation_period["tele"] == freq_pair].index[-1]
    oscillation_period = usage_cleaned.loc[start_index:end_index, "loc"].copy()

    count_loc1 = oscillation_period[oscillation_period == loc1].shape[0]
    count_loc2 = oscillation_period[oscillation_period == loc2].shape[0]
    
    while abs(count_loc1-count_loc2) < 5:
        start_index -= 1
        end_index += 1
        oscillation_period = usage_cleaned.loc[start_index:end_index, "loc"].copy()
        count_loc1 = oscillation_period[oscillation_period == loc1].shape[0]
        count_loc2 = oscillation_period[oscillation_period == loc2].shape[0]

    if count_loc1 > count_loc2:
        oscillation_period[oscillation_period == loc2] = loc1
    else:
        oscillation_period[oscillation_period == loc1] = loc2
        
    print(oscillation_period)

1782    3042
1783    3042
1784    3042
1785    3042
1786    3042
1787    1361
1788    1361
1789    1361
1790    1361
1791    1361
1792    1361
1793    1361
1794    1361
1795    1361
1796    1361
1797    1361
1798    1361
1799    1361
1800    1361
1801    1361
1802    1361
1803    1361
1804    1361
1805    1361
1806    1361
1807    1361
1808    1361
1809    1361
Name: loc, dtype: int64
1006    2318
1007    2318
1008    2318
1009    2318
1010    2318
1011    2318
1012    2318
1013    2318
1014    2318
1015    2318
1016    2318
1017    2318
1018    2318
1019    2318
1020    2318
1021    2318
1022    2318
1023    2318
1024    2318
Name: loc, dtype: int64
3577    7692
3578    7692
3579    7692
3580    7692
3581    7692
3582    7692
3583    7692
3584    7692
3585    7692
3586    7692
3587    7692
3588    7692
3589    7692
3590    7692
3591    7692
3592    7692
3593    7692
3594    7692
3595    7692
Name: loc, dtype: int64
time: 16 ms


## Turns oout this is a much faster as loc is much easier to manipulate the dataset
## Example

In [325]:
#before replace value
usage_cleaned.loc[1787:1804, "loc"]

1787    1361
1788    2304
1789    2304
1790    1361
1791    1361
1792    1361
1793    2304
1794    2304
1795    1361
1796    2304
1797    1361
1798    2304
1799    1361
1800    2304
1801    1361
1802    2304
1803    1361
1804    2304
Name: loc, dtype: int64

time: 0 ns


In [327]:
indextochange = usage_cleaned.loc[1787:1804, "loc"]
indextochange[indextochange == 2304] = 1361

time: 0 ns


In [329]:
usage_cleaned.loc[1787:1804]

,uid,timestamp,loc,app_id,traffic,date
1787,0,2016-04-22 14:26:31,1361,361,0.001,04-22
1788,0,2016-04-22 14:26:31,1361,361,0.003,04-22
1789,0,2016-04-22 14:26:41,1361,361,0.003,04-22
1790,0,2016-04-22 14:26:44,1361,41,0.012,04-22
1791,0,2016-04-22 14:26:45,1361,361,0.005,04-22
1792,0,2016-04-22 14:26:45,1361,41,0.001,04-22
1793,0,2016-04-22 14:26:45,1361,41,0.003,04-22
1794,0,2016-04-22 14:26:46,1361,41,0.007,04-22
1795,0,2016-04-22 14:26:48,1361,361,0.001,04-22
1796,0,2016-04-22 14:26:48,1361,361,0.002,04-22


time: 16 ms


# Ignore the following cells


# Replace the loc id with less count with the loc id with higher count in the frequent pair

In [74]:
# # Show the frequent pairs of user 13, gap less than 3 seconds, and occurence higher than 5 times (both directions count)
freq_pair = np.array(show_frequent_pair(0,10,5))
freq_pair

<ipython-input-72-08347122befa>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  user_uid['next_loc'] = user_uid['loc'].shift(-1).fillna(0).astype('int')
<ipython-input-72-08347122befa>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  user_uid['next_timestamp'] = user_uid['timestamp'].shift(-1)
<ipython-input-72-08347122befa>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/

array([[(2304, 1361), 7],
       [(1361, 2304), 7]], dtype=object)

time: 6.12 s


In [75]:
print(freq_pair[0][0])
print(freq_pair[1][0])

(2304, 1361)
(1361, 2304)
time: 0 ns


In [18]:
df_exp = usage_cleaned[(usage_cleaned["loc"] == freq_pair[0][0][0]) & (usage_cleaned["uid"] == str(0))]
df_exp

,uid,timestamp,loc,app_id,traffic,date
1787,0,2016-04-22 14:26:31,1361,361,0.001,04-22
1790,0,2016-04-22 14:26:44,1361,41,0.012,04-22
1791,0,2016-04-22 14:26:45,1361,361,0.005,04-22
1792,0,2016-04-22 14:26:45,1361,41,0.001,04-22
1795,0,2016-04-22 14:26:48,1361,361,0.001,04-22
1797,0,2016-04-22 14:26:49,1361,361,0.073,04-22
1799,0,2016-04-22 14:26:55,1361,361,0.002,04-22
1801,0,2016-04-22 14:26:56,1361,41,0.001,04-22
1803,0,2016-04-22 14:26:57,1361,41,0.003,04-22
1805,0,2016-04-22 14:27:00,1361,361,0.003,04-22


time: 281 ms


In [19]:
df_exp1 = usage_cleaned[(usage_cleaned["loc"] == freq_pair[0][0][1]) & (usage_cleaned["uid"] == str(0))]
df_exp1

,uid,timestamp,loc,app_id,traffic,date
1788,0,2016-04-22 14:26:31,2304,361,0.003,04-22
1789,0,2016-04-22 14:26:41,2304,361,0.003,04-22
1793,0,2016-04-22 14:26:45,2304,41,0.003,04-22
1794,0,2016-04-22 14:26:46,2304,41,0.007,04-22
1796,0,2016-04-22 14:26:48,2304,361,0.002,04-22
1798,0,2016-04-22 14:26:54,2304,361,0.006,04-22
1800,0,2016-04-22 14:26:55,2304,361,0.003,04-22
1802,0,2016-04-22 14:26:56,2304,361,0.002,04-22
1804,0,2016-04-22 14:26:57,2304,41,0.001,04-22


time: 297 ms


In [146]:
df_exp = pd.concat([df_exp, df_exp1])
df_exp

,uid,timestamp,loc,app_id,traffic,date
1787,0,2016-04-22 14:26:31,1361,361,0.001,04-22
1790,0,2016-04-22 14:26:44,1361,41,0.012,04-22
1791,0,2016-04-22 14:26:45,1361,361,0.005,04-22
1792,0,2016-04-22 14:26:45,1361,41,0.001,04-22
1795,0,2016-04-22 14:26:48,1361,361,0.001,04-22
1797,0,2016-04-22 14:26:49,1361,361,0.073,04-22
1799,0,2016-04-22 14:26:55,1361,361,0.002,04-22
1801,0,2016-04-22 14:26:56,1361,41,0.001,04-22
1803,0,2016-04-22 14:26:57,1361,41,0.003,04-22
1805,0,2016-04-22 14:27:00,1361,361,0.003,04-22


time: 16 ms


In [150]:
if (df_exp[(df_exp["loc"] == freq_pair[0][0][0]) & (df_exp["uid"] == str(0))].shape[0]) > (df_exp[(df_exp["loc"] == freq_pair[0][0][1]) & (df_exp["uid"] == str(0))].shape[0]):
    df_exp["loc"] = df_exp["loc"].replace(freq_pair[0][0][1], freq_pair[0][0][0])
df_exp

,uid,timestamp,loc,app_id,traffic,date
1787,0,2016-04-22 14:26:31,1361,361,0.001,04-22
1790,0,2016-04-22 14:26:44,1361,41,0.012,04-22
1791,0,2016-04-22 14:26:45,1361,361,0.005,04-22
1792,0,2016-04-22 14:26:45,1361,41,0.001,04-22
1795,0,2016-04-22 14:26:48,1361,361,0.001,04-22
1797,0,2016-04-22 14:26:49,1361,361,0.073,04-22
1799,0,2016-04-22 14:26:55,1361,361,0.002,04-22
1801,0,2016-04-22 14:26:56,1361,41,0.001,04-22
1803,0,2016-04-22 14:26:57,1361,41,0.003,04-22
1805,0,2016-04-22 14:27:00,1361,361,0.003,04-22


time: 15 ms
